### Tool Calling in LangChain

Tool Calling is the process where the LLM (language model) decides, during a conversation or task, that it needs to use a specific tool (function) and generates a structured output with:

- The name of the tool
- The arguments to call it with

The LLM does not actually run the tool it just suggests the tool and the input arguments. The actual execution is handled by LangChain or you.

In [1]:
from langchain.tools import tool

# Creating a Custom Tool
@tool
def multiply(a: float, b: float)->float:
    """Multiplies two numbers"""
    return a * b

c:\Users\gaurv\Downloads\STUDY\Generative AI\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq
# LLM Model Initialization using GROQ
llm = ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001AE950F2510>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AE950F3230>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
# Binding Tool with the LLM Model
llm_with_tool = llm.bind_tools([multiply])
llm_with_tool

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001AE950F2510>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AE950F3230>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiplies two numbers', 'parameters': {'properties': {'a': {'type': 'number'}, 'b': {'type': 'number'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [5]:
response = llm_with_tool.invoke("What is 7 times 8")

print(response) # We can see in the output that the LLM made a tool call to the multiply tool

content='' additional_kwargs={'tool_calls': [{'id': '3dr6zzpt4', 'function': {'arguments': '{"a":7,"b":8}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 221, 'total_tokens': 240, 'completion_time': 0.033133244, 'completion_tokens_details': None, 'prompt_time': 0.016604246, 'prompt_tokens_details': None, 'queue_time': 0.056327124, 'total_time': 0.04973749}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bfb6f-a474-7463-89fe-905066783718-0' tool_calls=[{'name': 'multiply', 'args': {'a': 7, 'b': 8}, 'id': '3dr6zzpt4', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 221, 'output_tokens': 19, 'total_tokens': 240}


In [6]:
llm_with_tool.invoke("What is 7 times 8").tool_calls[0]

{'name': 'multiply',
 'args': {'a': 7, 'b': 8},
 'id': '2fzgbd86v',
 'type': 'tool_call'}

### Tool Execution
Tool Execution is the step where the actual Python function (tool) is run using the input arguments that the LLM suggested during tool calling.

In [7]:
result = llm_with_tool.invoke("What is 7 times 8")

result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 7, 'b': 8},
 'id': '75w9pvf52',
 'type': 'tool_call'}

In [8]:
response = multiply.invoke(result.tool_calls[0])

response

ToolMessage(content='56.0', name='multiply', tool_call_id='75w9pvf52')

### LLM with Currency Conversion Tool

In [9]:
import requests
from langchain.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str)->float:
    """
    This function fetches the currency conversion factor between a base currency and a target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/8f2177314025283642253fb4/pair/{base_currency}/{target_currency}"

    response = requests.get(url)

    try:
        return response.json()['conversion_rate']
    except Exception as e:
        return f"Exception Caught : {e}"

@tool
def convert_currency(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg])->float:
    """
    Given a currency conversion rate, this function calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate

In [10]:
conversion_rate = get_conversion_factor.invoke({"base_currency" : "USD", "target_currency" : "INR"})

print(conversion_rate)

91.8161


In [11]:
target_currency = convert_currency.invoke({"base_currency_value" : 15, "conversion_rate" : conversion_rate})

print(target_currency)

1377.2415


In [44]:
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001AE9759D220>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AE9759C770>, model_name='llama-3.1-8b-instant', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [45]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert_currency])
llm_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001AE9759D220>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AE9759C770>, model_name='llama-3.1-8b-instant', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor between a base currency and a target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'conv

In [67]:
from langchain.messages import HumanMessage

messages = [HumanMessage(content="What is the conversion rate factor between USD and INR as of today, and based on that can you convert 10 usd to inr")]

messages

[HumanMessage(content='What is the conversion rate factor between USD and INR as of today, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [68]:
ai_message = llm_with_tools.invoke(messages)

ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'vv1nk9tz6', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'mqnqyh5h6', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":0.01}', 'name': 'convert_currency'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 356, 'total_tokens': 404, 'completion_time': 0.080153357, 'completion_tokens_details': None, 'prompt_time': 0.02484286, 'prompt_tokens_details': None, 'queue_time': 0.05392074, 'total_time': 0.104996217}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bfb73-e1ed-73a2-a2fb-9975d8959710-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'vv1nk9tz6

In [69]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'vv1nk9tz6',
  'type': 'tool_call'},
 {'name': 'convert_currency',
  'args': {'base_currency_value': 10, 'conversion_factor': 0.01},
  'id': 'mqnqyh5h6',
  'type': 'tool_call'}]

In [70]:
messages.append(ai_message)

In [71]:
for tool_call in ai_message.tool_calls:

    # Executing the first Tool to get the conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # Extracting Conversion Rate
        conversion_rate = float(tool_message1.content)
        # Appending message to message history list
        messages.append(tool_message1)
    
    # Executing the second tool using the fetched conversion rate from the first tool
    if tool_call['name'] == 'convert_currency':
        # Injecting the conversion rate to the tool call
        tool_call['args']['conversion_rate'] = conversion_rate
        # Invoking the tool to generate response
        tool_message2 = convert_currency.invoke(tool_call)
        # Appening response to the message history list
        messages.append(tool_message2)

In [72]:
messages

[HumanMessage(content='What is the conversion rate factor between USD and INR as of today, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'vv1nk9tz6', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'mqnqyh5h6', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":0.01}', 'name': 'convert_currency'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 356, 'total_tokens': 404, 'completion_time': 0.080153357, 'completion_tokens_details': None, 'prompt_time': 0.02484286, 'prompt_tokens_details': None, 'queue_time': 0.05392074, 'total_time': 0.104996217}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider

In [74]:
response = llm_with_tools.invoke(messages)

print(response.content)

The conversion rate factor between USD and INR as of today is 1 USD = 91.8161 INR. Therefore, 10 USD is equivalent to 918.161 INR.


### Summary
1. User says: "Convert 10 USD to INR."
2. LLM thinks: "I don't know the rate. First, let me call get_conversion_factor."
3. Tool result comes: 91.8161
4. LLM looks at result, THINKS again: "Now I know the rate, next I should call convert with 10 and 91.8161."
5. Tool result comes: 918.161 INR
6. LLM summarizes: "10 USD is 918.161 INR at current rate."